In [1]:
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import scipy.io, cv2

# Find images path
images_paths = glob('../Dataset_Final/jpg_images/*/*')

# Fulfill data
X_ = []
y_ = []
for path in images_paths[0:10]:
    scene = path.split('/')[-2]
    
    # Input data (images)
    img = plt.imread(path)
    # Image resize
    if 1:
        img = cv2.resize(img, dsize = None,fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
    
    X_.append(img)
    
    # Output data (NLFs)
    nlf_path = '../Dataset_Final/NLFs/NLF_' + scene + '.mat'
    mat = scipy.io.loadmat('../Dataset_Final/NLFs/NLF_' + scene + '.mat')
    NLF = mat["NLF"][0]
    y_.append(NLF)
    
#(X_train, y_train), (X_test, y_test) = ''

In [2]:
X_ = np.asarray(X_)
y_ = np.asarray(y_)


In [3]:
print(np.shape(X_))

(10, 1536, 2304, 3)


### Crear modelo

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten, concatenate, AveragePooling2D, Activation, BatchNormalization
from keras.models import Model
from SpatialPyramidPooling import SpatialPyramidPooling
# First, define the vision modules
input_imgs = Input(shape=(None, None, 3))

# Model
def get_model_column(ks_1, ks_2):
    x = Conv2D(64, kernel_size=(ks_1,ks_1), strides = 2, activation=None)(input_imgs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D((2, 2))(x)
    x = Conv2D(64, kernel_size=(ks_2,ks_2), activation=None)(x) # conv2_a
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D((2, 2))(x) # pool2_a
    x = Conv2D(64, kernel_size=(ks_2,ks_2), activation=None)(x) # conv2_a
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, kernel_size=(ks_2,ks_2), activation=None)(x) # conv2_a
    x = BatchNormalization()(x)
    out = Activation('relu')(x)
    #out1 = Flatten()(x)
    return out

vision_model1 = Model(input_imgs, get_model_column(5, 3))
vision_model2 = Model(input_imgs, get_model_column(7, 5))
vision_model3 = Model(input_imgs, get_model_column(9, 7))

# Then define the input-apart model
input_a = Input(shape=(None, None, 3))
input_b = Input(shape=(None, None, 3))
input_c = Input(shape=(None, None, 3))

# The vision model will be shared, weights and all
out_a = vision_model1(input_a)
out_b = vision_model2(input_b)
out_c = vision_model3(input_c)

concatenated = concatenate([out_a, out_b, out_c],axis=1)
#out = Dense(1, activation='sigmoid')(concatenated)
merge_abc = SpatialPyramidPooling([1, 2, 4])(concatenated)
merge_abc = Dense(2048, activation='relu')(merge_abc)
merge_abc = Dense(256*3, activation='relu')(merge_abc)

model_abc = Model([input_a, input_b, input_c], merge_abc)

In [7]:
# Make model_a non trainable
model_abc.layers[3].trainable = False
model_abc.layers[4].trainable = False

for layer in model_abc.layers:
    print(layer.trainable)

False
False
False
False
False
True
True
True
True
True


In [5]:
model_abc.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, None, None, 6 116672      input_2[0][0]                    
__________________________________________________________________________________________________
model_2 (M

### Training

In [8]:
from keras import optimizers

sgd = optimizers.SGD(lr=0.001, decay=0.0001, momentum=0.9, nesterov=True)

# compile model (abc)
model_abc.compile(optimizer= sgd, loss='mse')

In [ ]:
model_abc.fit(x=[X_,X_,X_], y=y_, batch_size=10, epochs=10, validation_split=0, shuffle=True)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10


In [1]:
(model_abc.predict([[X_[0]],[X_[0]],[X_[0]]]))

NameError: name 'model_abc' is not defined

In [7]:
from keras.utils import plot_model

plot_model(model_abc, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
